In [1]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors

sc.stop()
conf = SparkConf().set("spark.executor.instances", 4).set("spark.driver.maxResultSize", "4g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)


query = '''
select * from user_kposminin.more_features_coldcalls_2weeks_20160414
'''

data = hc.sql(query) 

train_data = data \
    .filter("ymd < '2016-04-12'") \
    .toPandas() \
    .dropna()
    
test_data = data \
    .filter("ymd >= '2016-04-12'") \
    .toPandas() \
    .dropna()

In [2]:
train_data.head()


,ymd,phone_num,approve,full_app,max_score,min_score,avg_score,cnt,hits,min_avg_hour,max_avg_hour
0,2016-04-01,89000165602,0,0,-8.149877,-15.405658,-9.957737,758,4926,0,23
1,2016-04-01,89000262616,0,0,-8.854922,-11.085176,-10.095812,50,114,8,11
2,2016-04-01,89000288815,0,0,-8.635742,-12.568947,-9.575737,406,2171,7,23
3,2016-04-01,89000415018,0,0,-8.345416,-12.836229,-9.820509,212,2315,0,23
4,2016-04-01,89000865324,0,0,-8.251351,-12.597713,-9.680008,314,1767,1,23


In [3]:
import sklearn
import xgboost as xgb
clfXGB = xgb.XGBClassifier()
clfXGB.fit(X = train_data.iloc[:,4:],y = train_data['full_app'])
print('{0} AUCROC: {1}'.format('XGBoost',sklearn.metrics.roc_auc_score(
                y_true = test_data['full_app'], 
                y_score = [e[1] for e in clfXGB.predict_proba(test_data.iloc[:,4:])]
    )))

XGBoost AUCROC: 0.554252978211


In [5]:
train_data['approve'].mean()

0.045550492772189304

In [15]:
sc.stop()